# Tutorial 08 - Frame Extraction and Gaze Visualization

Pupil Core distinguishes between `System Time` and `Pupil Time`, measured in seconds.

`System Time` is the current time of the device running Pupil Core software and uses the Unix epoc, while `Pupil Time` has an arbitrary that can be used to synchronize the clock between multiple devices.

Since the exported data (pupil, gaze, fixations, blinks, surface, etc.) uses timestamps in `Pupil Time`, it is often desireable to convert these timestamps into Unix timestamps (`System Time`), or into `datetime` objects in Python.

This tutorial shows how to easily perform the conversion and save the data in a new file.

---

> To execute this notebook, download the [sample recording](https://drive.google.com/file/d/1vzjZkjoi8kESw8lBnsa_k_8hXPf3fMMC/view?usp=sharing). Unzip and move it into the `recordings` directory for this repository.

In [ ]:
import pathlib
import json

import numpy as np
import pandas as pd

pd.options.display.float_format = '{:}'.format

DATAFRAME_HEAD_COUNT = 3

First, we define the path to the recording directory, as well as the export directory within the recording.

In [ ]:
rec_dir = pathlib.Path(".").joinpath("recordings").joinpath("sample_recording_v2").absolute()
assert rec_dir.is_dir(), "Please download the sample recording into 'recordings' directory."
rec_dir

In [ ]:
export_dir = rec_dir.joinpath("exports").joinpath("000")
assert export_dir.is_dir(), "Please create at least one export."
export_dir

The recording contains a meta-data file (`info.player.json`) which provide essential information about the recording itself, as well as the context in which it was made. More information about the format can be found [here](https://github.com/pupil-labs/pupil/blob/master/pupil_src/shared_modules/pupil_recording/README.md).

In [ ]:
with rec_dir.joinpath("info.player.json").open() as file:
    meta_info = json.load(file)

meta_info

Using the start time of the recording in `System Time` (`start_time_system_s` field) and in `Pupil Time` (`start_time_synced_s` field), we calculate the offset which will be applied to timestamps in other data files to convert them to Unix timestamps.

In [ ]:
start_timestamp_unix = meta_info["start_time_system_s"]
start_timestamp_pupil = meta_info["start_time_synced_s"]
start_timestamp_diff = start_timestamp_unix - start_timestamp_pupil

## Pupil Positions Timestamps

The code bellow implements the following steps:
- Load the `pupil_positions.csv` file from the export directory into a Pandas dataframe
- Convert the `pupil_timestamp` column values to Unix timestamps (new `pupil_timestamp_unix` column)
- Convert the `pupil_timestamp` column values to datetime objects (new `pupil_timestamp_datetime` column)
- Save the updated dataframe into `pupil_positions_unix_datetime` file in the export directory

In [ ]:
pupil_positions_df = pd.read_csv(export_dir.joinpath("pupil_positions.csv"))
pupil_positions_df.head(DATAFRAME_HEAD_COUNT)

In [ ]:
pupil_positions_df["pupil_timestamp_unix"] = pupil_positions_df["pupil_timestamp"] + start_timestamp_diff
pupil_positions_df.head(DATAFRAME_HEAD_COUNT)

In [ ]:
pupil_positions_df["pupil_timestamp_datetime"] = pd.to_datetime(pupil_positions_df["pupil_timestamp_unix"], unit="s")
pupil_positions_df.head(DATAFRAME_HEAD_COUNT)

In [ ]:
pupil_positions_df.to_csv(export_dir.joinpath("pupil_positions_unix_datetime.csv"))

Bellow, the same steps are used to convert and save Unix and datetime timestamps for gaze and fixation data

## Gaze Positions Timestamps

In [ ]:
gaze_positions_df = pd.read_csv(export_dir.joinpath("gaze_positions.csv"))
gaze_positions_df["gaze_timestamp_unix"] = gaze_positions_df["gaze_timestamp"] + start_timestamp_diff
gaze_positions_df["gaze_timestamp_datetime"] = pd.to_datetime(gaze_positions_df["gaze_timestamp_unix"], unit="s")
gaze_positions_df.to_csv(export_dir.joinpath("gaze_positions_unix_datetime.csv"))
gaze_positions_df.head(DATAFRAME_HEAD_COUNT)

## Fixations Timestamps

In [ ]:
fixations_df = pd.read_csv(export_dir.joinpath("fixations.csv"))
fixations_df["start_timestamp_unix"] = fixations_df["start_timestamp"] + start_timestamp_diff
fixations_df["start_timestamp_datetime"] = pd.to_datetime(fixations_df["start_timestamp_unix"], unit="s")
fixations_df.to_csv(export_dir.joinpath("fixations_unix_datetime.csv"))
fixations_df.head(DATAFRAME_HEAD_COUNT)

## Surfaces Timestamps

In [ ]:
surfaces_dir = export_dir.joinpath("surfaces")
assert surfaces_dir.is_dir(), "Please add at least one surface to the export."
surfaces_dir

To aid in converting multiple files, some of which have more than one column with timestamp values, the `convert_and_save_timestamps` function is defined bellow, which replicates the steps previously described.

In [ ]:
def convert_and_save_timestamps(input_path, column_names, timestamp_offset=start_timestamp_diff):
    
    output_path = input_path.with_name(input_path.stem + "_unix_datetime").with_suffix(input_path.suffix)

    df = pd.read_csv(input_path)

    for column_name in column_names:
        unix_column_name = column_name + "_unix"
        datetime_column_name = column_name + "_datetime"

        df[unix_column_name] = df[column_name] + timestamp_offset
        df[datetime_column_name] = pd.to_datetime(df[unix_column_name], unit="s")

    df.to_csv(output_path)

    return df.head(DATAFRAME_HEAD_COUNT)

In [ ]:
convert_and_save_timestamps(
    input_path=surfaces_dir.joinpath("surface_events.csv"),
    column_names=["world_timestamp"]
)

In [ ]:
convert_and_save_timestamps(
    input_path=surfaces_dir.joinpath("surf_positions_Cover.csv"),
    column_names=["world_timestamp"]
)

In [ ]:
convert_and_save_timestamps(
    input_path=surfaces_dir.joinpath("gaze_positions_on_surface_Cover.csv"),
    column_names=["world_timestamp", "gaze_timestamp"]
)

In [ ]:
convert_and_save_timestamps(
    input_path=surfaces_dir.joinpath("fixations_on_surface_Cover.csv"),
    column_names=["world_timestamp", "start_timestamp"]
)